In [92]:
# Framework:
#
# 1. Individual stock:
#      ---- GENERAL INPUT: only one sticker
#   1.1. Parameter estimates:
#      - plot_1: mean (2/5/10 years rolling windows in one plot)
#      - plot_2: volatility (2/5/10 years rolling windows in one plot)
#   1.2. VaR/ES
#      - input: VaR prob, ES prob, rolling windows*, horizon, method**
#      - plot: VaR and ES in one plot
# 2. Portfolio: (portfolio lognormal/position normal)
#      ---- GENERAL INPUT: stickers, initial investment, weights
#   2.1. Parameter estimates:
#      - plot_1: mean (2/5/10 years rolling windows in one plot)
#      - plot_2: volatility (2/5/10 years rolling windows in one plot)
#   2.2. VaR/ES (ASSUMPTION: portfolio following GBM)
#      - input: VaR prob, ES prob, rolling window*, horizon, method**
#      - plot: VaR and ES in one plot
#
#    * choose from 2/5/10 because parameters already computed above
#   ** choose from historical/parametric/MC
#  *** historical method: assumes relative price changes (log returns)
#      parametric/MC methods: assumes portfolio follow GBM

In [ ]:
# TODO:
#
# 1. Portfolio parameters (hw6)
# 2. Portfolio VaR/ES (hw6)
# 3. Historical method (hw7)
# 4. Monte Carlo (hw8)
# 5. Plot

In [ ]:
# import packages

import pandas_datareader.data as web
import datetime
import pandas as pd
import numpy as np
import scipy.stats as stat
import dateutil.relativedelta

from __future__ import division

In [85]:
# set up the parameters
position_date = datetime.datetime(2010, 12, 1)
end_date = datetime.datetime(2016, 12, 1)

v0 = 10000    # initial investment

tickers_string = "AAPL,AMZN"   # a string; as an example, tickers separated by comma.
tickers_list = tickers_string.split(",")  # split the string by the commas into a list of strings
tickers_num = len(tickers_list)

weight_string = "0.5,0.5"
weight_list = map(float, weight_string.split(","))  
# split the string by the commas, and map each string in the resulting list into a float

VaR_prob = 0.99
ES_prob = 0.975
window = 2    # Using x years historical data for estimation
window_days = window * 252   
horizon_days = 5     # x days VaR/ES
horizon = horizon_days/252

start_date = position_date - dateutil.relativedelta.relativedelta(years = window)      # fetch data starting from start_date

In [86]:
d={}
for ticker in tickers_list:
    d["{0}".format(ticker)] = web.DataReader(ticker, 'yahoo', start_date, end_date)['Adj Close'].rename(ticker)
df = pd.DataFrame(d).sort_index(ascending = False)       
# create a pandas dataframe where each column is the Adjusted Close price for the respective ticker

In [87]:
# Calculate estimated parameters for GBM based on x year (in days) rolling windows
def gbm_est(prices, window_days):
    rtn = -np.diff(np.log(prices))
    rtnsq = rtn * rtn
    mubar = list(reversed(np.convolve(rtn, np.ones((window_days,))/window_days, mode='valid')))
    x2bar = list(reversed(np.convolve(rtnsq, np.ones((window_days,))/window_days, mode='valid')))
    var = x2bar - np.square(mubar)
    sigmabar = np.sqrt(np.maximum(var, np.zeros(len(var))))
    sigma = sigmabar / np.sqrt(1/252)
    mu = np.array(mubar)*252 + np.square(sigma)/2
    return rtn, mu, sigma, np.array(mubar), sigmabar

In [88]:
# test with AAPL 
rtn, mu, sigma, mubar, sigmabar = gbm_est(df["AAPL"], window_days)

In [ ]:
df['AAPL'][position_date]

In [ ]:
rtn   # compare with values computed in Matlab

In [ ]:
mu   # compare with values computed in Matlab

In [ ]:
sigma   # compare with values computed in Matlab

In [ ]:
mubar   # compare with values computed in Matlab

In [ ]:
sigmabar   # compare with values computed in Matlab

In [ ]:
# Calculate t-day VaR with confidence degree p assuming gbm, --parametric method
# Using parameter estimated above 
def gbm_VaR(v0, mu, sigma, p, t):
    VaR = v0 - v0 * np.exp(sigma * np.sqrt(t) * stat.norm.ppf(1-p) + (mu - np.square(sigma)/2) * t)
    return VaR

In [ ]:
VaR = gbm_VaR(v0, mu, sigma, VaR_prob, horizon)

In [ ]:
VaR   # compare with values computed in Matlab

In [ ]:
# Calculate t-day ES with confidence degree p assuming gbm, --parametric method\
# Using parameter estimated above 
def gbm_ES(v0, mu, sigma, p, t):
    ES = v0 * (1 - np.array(stat.norm.cdf(stat.norm.ppf(1-p) - np.sqrt(t)*sigma)) * np.array(np.exp(mu*t)/(1-p)))
    return ES

In [ ]:
ES = gbm_ES(v0, mu, sigma, ES_prob, horizon)

In [ ]:
ES   # compare with values computed in Matlab

In [89]:
shares = np.round(np.divide(v0 * np.array(weight_list), np.array(df.ix[position_date])))

In [90]:
shares

array([ 121.,   28.])